In [33]:
import ee
ee.Initialize()

In [34]:
inputCSV = ee.FeatureCollection('users/ccrs2fy2020/features')
inputCSV = inputCSV.toList(count=1000000)

In [35]:
def extractBands(feature):
    feature = ee.Feature(feature)
    return feature.toArray(properties=['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'A1', 'A2', 'A3']).toList()

def extractVI(feature):
    feature = ee.Feature(feature)
    return feature.toArray(properties=['FCOVER']).toList()

In [36]:
inputList = inputCSV.map(extractBands)
outputList = inputCSV.map(extractVI)

In [37]:
inputs = ee.Array(inputList)
inputDimensions = inputs.length()
n = inputDimensions.get([0])
m = inputDimensions.get([1])

outputs = ee.Array(outputList)
initialResiduals = ee.Array(ee.List.repeat([0], n))

inputData = ee.Dictionary.fromLists(['inputs', 'outputs', 'currentResiduals'], [inputs, outputs, initialResiduals])

In [38]:
def LARSregression(placeHolder, inputData):
    inputData = ee.Dictionary(inputData)
    X = inputData.getArray('inputs')
    y = inputData.getArray('outputs')
    currentResiduals = inputData.getArray('currentResiduals')

    X_t = X.matrixTranspose()
    c = X_t.matrixMultiply(y.subtract(currentResiduals))
    c_abs = c.abs()
    C_max = c_abs.get(c_abs.argmax())

    maxLocs = c_abs.gte(C_max)
    signs = c.divide(c_abs)
    
    signs_j = maxLocs.multiply(signs).matrixTranspose()
    signs_jc = signs_j.subtract(1).multiply(-1)

    A = ee.List(ee.Array([ee.List.sequence(0, m.subtract(1))]).mask(signs_j).toList().get(0))
    A_c = ee.List(ee.Array([ee.List.sequence(0, m.subtract(1))]).mask(signs_jc).toList().get(0))

    signMatrix_j = signs_j.repeat(0, n)

    X_A = X.multiply(signMatrix_j).mask(signs_j)

    G_A = X_A.matrixTranspose().matrixMultiply(X_A)

    j = X_A.length().get([1])

    V1_A = ee.Array(ee.List.repeat([1], j))

    G_Ai = G_A.matrixInverse()

    A_A = V1_A.matrixTranspose().matrixMultiply(G_Ai).matrixMultiply(V1_A).get([0,0]).pow(-0.5)

    w_A = G_Ai.matrixMultiply(V1_A).multiply(A_A)

    u_A = X_A.matrixMultiply(w_A)

    a = X_t.matrixMultiply(u_A)

   
    def computeGamma(index_j):
        minus_j = C_max.subtract(c.get([index_j, 0])).divide(A_A.subtract(a.get([index_j, 0])))
        plus_j = C_max.add(c.get([index_j, 0])).divide(A_A.add(a.get([index_j, 0])))

        gammaRRay = ee.Array([minus_j, plus_j]);
        gammaRRay = gammaRRay.mask(gammaRRay.gte(0))
        gammaRRay = gammaRRay.multiply(-1)

        return gammaRRay.get(gammaRRay.argmax())
    

    gammaRRay = ee.Array([A_c.map(computeGamma)])
    gamma = gammaRRay.get(gammaRRay.argmax()).multiply(-1)

    ret = currentResiduals.add(u_A.multiply(gamma))
    outputData = ee.Dictionary.fromLists(['inputs', 'outputs', 'currentResiduals'], [X, y, ret])

    return outputData;

In [39]:
numIterations = ee.List.repeat(0, m)
finalOutput = ee.Dictionary(numIterations.iterate(LARSregression, inputData))
finalResiduals = finalOutput.getArray('currentResiduals')

coefficients = inputs.matrixSolve(finalResiduals)

In [40]:
coefficients.getInfo()

[[-6.728316485003986e-13],
 [1.3324645866039069e-13],
 [9.138882060133785e-14],
 [1.6419527952436233e-13],
 [0.4487669198735426],
 [0.8309103773562737],
 [-5.43280334250975e-14],
 [7.614961124135065e-14],
 [2.9394488380003407],
 [5.94331346496581e-17],
 [3.4098272887961243e-17]]